<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-08-29 19:29:23


In [20]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [21]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*10)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [39]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [40]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [24]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
1.0    66
0.0    65
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    29
V40     26
V200     2
VR       1
SS       1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    125
0      7
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1.0    30
0.0    25
Name: StarStock, dtype: int64
-------


In [25]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [26]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [27]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='10y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

3MINDIA
ABBOTINDIA
ACC
ADANIENT
ADANIGREEN
ADANIPORTS
AKZOINDIA
ALKYLAMINE
ANGELONE
APOLLOHOSP
ASIANPAINT
ASTRAZEN
AUBANK
AUROPHARMA
AVANTIFEED
AXISBANK
BAJAJ-AUTO
BAJAJHIND
BAJAJHLDNG
BAJFINANCE
BHARTIARTL
BIOCON
BOSCHLTD
BPCL
BRITANNIA
CANBK
CAPLIPOINT
CERA
CHOLAFIN
CIPLA
COALINDIA
COLPAL
DABUR
DEEPAKNTR
DIVISLAB
DIXON
DLF
DRREDDY
EQUITAS
EQUITASBNK
ERIS
FEDERALBNK
FINCABLES
GAIL
GLAXO
GODREJCP
GODREJIND
GRASIM
HAVELLS
HCLTECH
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDPETRO
HINDUNILVR
HONAUT
IBREALEST
ICICIBANK
ICICIPRULI
IGL
INDIANB
INDIGO
INDUSTOWER
IOC
ISEC
ITC
JCHAC
JINDALSTEL
JSWSTEEL
KANSAINER
LALPATHLAB
LT
LUPIN
M&M
MARICO
MARUTI
MCDOWELL-N
MCX
MOTILALOFS
MUTHOOTFIN
NAM-INDIA
NATIONALUM
NAUKRI
NESTLEIND
NETWORK18
NTPC
OFSS
ONGC
PEL
PFIZER
PGHH
PIIND
POLYCAB
POWERGRID
PVRINOX
RADICO
RELIANCE
SANOFI
SBICARD
SBILIFE
SFL
SHREECEM
SIEMENS
SIS
SUNPHARMA
SUZLON
SYMPHONY
TASTYBITE
TATACONSUM
TATAELXSI
TATAMOTORS
TATAPOWER
TATASTEEL
TECHM
TIPSINDLTD
TITAN
TORNTPHARM
TTKPRESTIG
UJJIVAN
UJJIVANSF

In [28]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-31 00:00:00+05:30,19754.0,18180.0,8.66,^NSEI,19979.0,63.0
2023-08-07 00:00:00+05:30,19597.0,18240.0,7.44,^NSEI,19979.0,55.0
2023-08-14 00:00:00+05:30,19435.0,18289.0,6.26,^NSEI,19979.0,48.0
2023-08-22 00:00:00+05:30,19396.0,18327.0,5.84,^NSEI,19979.0,47.0
2023-08-29 00:00:00+05:30,19343.0,18358.0,5.36,^NSEI,19979.0,46.0


In [29]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [60]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [61]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Target']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [62]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-19.26,46.0,126.00,7.81,22.8,NaN,NaN,1,1.0,0.0,1,0.0,L,NaN,205.0
MOTILALOFS.NS,28.59,56.0,9.13,13.00,15.6,V40N,NaN,1,1.0,0.0,NaN,0.0,L,FINANCE,67.0
TATAELXSI.NS,8.93,63.0,451.00,60.00,39.3,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,44.0
SBICARD.NS,2.05,37.0,35.00,13.00,25.7,NaN,NaN,1,1.0,0.0,NaN,1.0,M,NaN,37.0
KANSAINER.NS,15.26,57.0,49.30,14.50,10.7,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,32.0
CANBK.NS,8.50,47.0,4.63,5.33,14.8,NaN,NaN,1,1.0,0.0,1,1.0,H,BANKS,30.0
GODREJIND.NS,15.35,62.0,18.60,9.75,11.2,NaN,NaN,1,1.0,0.0,NaN,0.0,L,NaN,30.0
VINATIORGA.NS,-4.59,48.0,42.50,30.40,22.6,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,29.0
ASIANPAINT.NS,7.76,52.0,67.60,35.20,27.7,V40,BTT,1,1.0,1.0,1,1.0,H,PAINTS,29.0


In [46]:
# top 10 prospects for SIP based on 200 DMA and RSI
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,200_SMA,Dev%_200,Max,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Symbol,Criteria,Strategy,LatestQtr,Target,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,
RELIANCE.NS,2420.0,2475.0,-2.21,2832.0,34.0,25.3,10.00,8.94,RELIANCE,V40,ATH,1,2832.0,1.0,1.0,1,1.0,H,REFINERIES,17.0
VEDL.NS,237.0,262.0,-9.54,295.0,36.0,11.8,23.80,20.40,VEDL,NaN,NaN,1,295.0,0.0,0.0,NaN,NaN,L,NaN,24.0
DABUR.NS,550.0,551.0,-0.15,642.0,38.0,56.8,23.30,19.50,DABUR,V40,NaN,1,642.0,1.0,0.0,1,1.0,H,FMCG,17.0
HINDUNILVR.NS,2526.0,2568.0,-1.63,2757.0,39.0,57.3,26.60,20.50,HINDUNILVR,V40,NaN,1,2757.0,1.0,0.0,NaN,1.0,H,FMCG,9.0
SFL.NS,1103.0,1168.0,-5.60,1972.0,39.0,51.3,13.60,12.50,SFL,V40N,NaN,1,1972.0,0.0,1.0,NaN,NaN,M,NaN,79.0
PFIZER.NS,3839.0,3957.0,-2.98,6002.0,43.0,114.0,27.20,19.80,PFIZER,V40,NaN,1,6002.0,0.0,0.0,NaN,0.0,L,PHARMA,56.0
ADANIGREEN.NS,974.0,1207.0,-19.26,2970.0,46.0,126.0,7.81,22.80,ADANIGREEN,NaN,NaN,1,2970.0,1.0,0.0,1,0.0,L,NaN,205.0
UPL.NS,598.0,693.0,-13.77,818.0,46.0,16.5,15.70,13.40,UPL,NaN,NaN,1,818.0,0.0,0.0,NaN,NaN,L,NaN,37.0
YESBANK.NS,17.0,17.0,-0.88,383.0,46.0,63.1,4.94,1.99,YESBANK,NaN,NaN,1,383.0,0.0,0.0,NaN,0.0,L,NaN,2153.0


In [49]:
# star + v40/v40n prospects (for upside)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 6


,Close,200_SMA,Dev%_200,Max,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Symbol,Criteria,Strategy,LatestQtr,Target,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,
MOTILALOFS.NS,880.0,684.0,28.59,1473.0,56.0,9.13,13.0,15.6,MOTILALOFS,V40N,NaN,1,1473.0,1.0,0.0,NaN,0.0,L,FINANCE,67.0
TATAELXSI.NS,7383.0,6778.0,8.93,10602.0,63.0,451.00,60.0,39.3,TATAELXSI,V40N,NaN,1,10602.0,1.0,1.0,NaN,NaN,M,NaN,44.0
KANSAINER.NS,330.0,286.0,15.26,436.0,57.0,49.30,14.5,10.7,KANSAINER,V40N,NaN,1,436.0,1.0,1.0,NaN,NaN,M,NaN,32.0
ASIANPAINT.NS,3286.0,3049.0,7.76,3544.0,52.0,67.60,35.2,27.7,ASIANPAINT,V40,BTT,1,4250.0,1.0,1.0,1,1.0,H,PAINTS,29.0
VINATIORGA.NS,1818.0,1906.0,-4.59,2342.0,48.0,42.50,30.4,22.6,VINATIORGA,V40N,NaN,1,2342.0,1.0,1.0,NaN,NaN,M,NaN,29.0
BAJFINANCE.NS,7293.0,6549.0,11.37,7860.0,57.0,35.80,11.8,23.5,BAJFINANCE,V40,BTT,1,9150.0,1.0,1.0,1,1.0,H,FINANCE,25.0


In [50]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Symbol,Criteria,Strategy,LatestQtr,Target,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,
MARICO.NS,562.0,514.0,9.44,585.0,53.0,53.6,43.00,35.8,MARICO,V40,NaN,1,585.0,1.0,0.0,NaN,1.0,M,FMCG,4.0
MCDOWELL-N.NS,1003.0,860.0,16.68,1039.0,53.0,59.2,19.90,17.5,MCDOWELL-N,V40N,NaN,1,1039.0,1.0,0.0,NaN,0.0,L,BREWERIES,4.0
POLYCAB.NS,5021.0,3286.0,52.79,5135.0,66.0,52.0,27.10,20.0,POLYCAB,V40N,NaN,1,5135.0,1.0,1.0,NaN,1.0,M,CABLES,2.0
TITAN.NS,3079.0,2669.0,15.38,3134.0,61.0,84.9,25.10,30.8,TITAN,V40,NaN,1,3134.0,1.0,0.0,NaN,1.0,H,JEWELLERY,2.0
AXISBANK.NS,981.0,909.0,7.89,990.0,60.0,13.2,6.16,14.5,AXISBANK,V40,NaN,1,990.0,1.0,0.0,1,1.0,M,BANKS,1.0
CERA.NS,9243.0,6551.0,41.09,9338.0,76.0,52.2,25.80,19.4,CERA,V40N,NaN,1,9338.0,1.0,1.0,1,1.0,H,CERAMICS,1.0
ICICIBANK.NS,968.0,898.0,7.77,982.0,55.0,18.2,6.35,17.2,ICICIBANK,V40,NaN,1,982.0,1.0,0.0,NaN,1.0,H,BANKS,1.0
3MINDIA.NS,31581.0,24345.0,29.72,31581.0,72.0,71.8,31.40,23.4,3MINDIA,V40N,NaN,1,31581.0,1.0,1.0,NaN,NaN,M,NaN,0.0
CAPLIPOINT.NS,1059.0,749.0,41.34,1063.0,66.0,20.4,26.80,22.4,CAPLIPOINT,V40N,NaN,1,1063.0,1.0,1.0,1,NaN,M,NaN,0.0


In [51]:
# prospects for interactive analysis

df_prospects

,Close,200_SMA,Dev%_200,Max,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Symbol,Criteria,Strategy,LatestQtr,Target,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,
3MINDIA.NS,31581.0,24345.0,29.72,31581.0,72.0,71.8,31.40,23.40,3MINDIA,V40N,NaN,1,31581.0,1.0,1.0,NaN,NaN,M,NaN,0.0
ABBOTINDIA.NS,23415.0,21560.0,8.60,24556.0,49.0,48.2,41.10,31.60,ABBOTINDIA,V40,BTT,1,27500.0,1.0,1.0,1,NaN,M,PHARMA,17.0
ACC.NS,1982.0,1986.0,-0.21,2720.0,57.0,43.8,10.00,7.06,ACC,NaN,NaN,0,2720.0,0.0,0.0,1,0.0,L,CEMENT,37.0
ADANIENT.NS,2506.0,2604.0,-3.75,4161.0,50.0,103.0,10.10,9.63,ADANIENT,NaN,NaN,1,4161.0,0.0,0.0,1,1.0,M,NaN,66.0
ADANIGREEN.NS,974.0,1207.0,-19.26,2970.0,46.0,126.0,7.81,22.80,ADANIGREEN,NaN,NaN,1,2970.0,1.0,0.0,1,0.0,L,NaN,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,665.0,645.0,3.09,743.0,53.0,57.0,27.80,29.00,VIPIND,V40N,NaN,1,743.0,0.0,1.0,NaN,NaN,M,NaN,12.0
VOLTAS.NS,849.0,812.0,4.54,1316.0,65.0,85.0,9.74,4.42,VOLTAS,NaN,NaN,1,1316.0,0.0,0.0,NaN,0.0,L,NaN,55.0
WHIRLPOOL.NS,1634.0,1419.0,15.18,2658.0,80.0,98.7,8.78,6.37,WHIRLPOOL,V40,NaN,1,2658.0,0.0,0.0,NaN,1.0,M,NaN,63.0


In [63]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [64]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-31 00:00:00+05:30,679.0,606.0,12.09,BERGEPAINT.NS,854.0,57.0
2023-08-07 00:00:00+05:30,707.0,608.0,16.26,BERGEPAINT.NS,854.0,66.0
2023-08-14 00:00:00+05:30,699.0,611.0,14.46,BERGEPAINT.NS,854.0,58.0
2023-08-22 00:00:00+05:30,706.0,614.0,14.93,BERGEPAINT.NS,854.0,61.0
2023-08-29 00:00:00+05:30,720.0,617.0,16.77,BERGEPAINT.NS,854.0,65.0
